In [5]:
import pandas as pd
from tabulate import tabulate
from fetch_process_function import fetch_and_save_erc20_transfers, process_transactions, calculate_pnl, process_and_save_address
'''
data_process.ipynb和data_fetch.ipynb同样功能，但是通过封装函数实现了给日期地址等参数就可以返回数据，更加简洁
添加了pnl的计算，我们这里只计算第一种交易的pnl，其余的暂时忽略不计。
最后对于address也进行一个处理
'''
# ADDRESS = "0x5be9a4959308A0D0c7bC0870E319314d8D957dBB"
ADDRESS = "0x2c89a2ee92b9870f55989b4132a58c0e85222d86"
# ADDRESS = "0x6c2a355929ee1262305e385ad49b84fe5f5a4777"

START_DATE = "2024-01-01 08:00" # Singapore Time，the function will convert it to UTC
END_DATE = "2025-01-01 08:00"
OUTPUT_FILE = "C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result"
API_KEY = "VQAIR728IM4Z8RZKPYBR4ESM5I3WBZK2C1"
BASE_TOKENS = {"USDT", "USDC", "USDE"}

In [6]:
transfers_df = fetch_and_save_erc20_transfers(
        address=ADDRESS,
        start_date=START_DATE,
        end_date=END_DATE,
        output_file=OUTPUT_FILE,
        offset=1000,
    )
transfers_df = pd.read_csv(OUTPUT_FILE + "/erc20_transfers.csv")
record_df_1, record_df_2, record_df_3 = process_transactions(transfers_df, OUTPUT_FILE, ADDRESS, BASE_TOKENS)
# 这里我为了显示简介把process_transactions中print结果的注释掉了 如果需要可以取消注释
if not record_df_1.empty:
    unique_tokens_count = record_df_1["token_symbol"].nunique()
    unique_tokens = record_df_1["token_symbol"].unique()
    results = []

    for token in unique_tokens:
        token_df = record_df_1[record_df_1["formatted_record"].str.contains(fr"\b{token}\b", regex=True)]
        print(f"{token}: {len(token_df)} records")

        pnl_df = calculate_pnl(token_df)
        print(tabulate(pnl_df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))   
        total_pnl = pnl_df["pnl"].sum()

        results.append({"token": token, "total_pnl": total_pnl})

    pnl_summary_df = pd.DataFrame(results)

    print(tabulate(pnl_summary_df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))
else:
    print("No transactions found")


2025-01-10 17:47:01,649 - INFO - Start block: 18908894, End block: 21525891
2025-01-10 17:47:01,650 - INFO - Fetching ERC20 transfers from block 18908894 to 21525891...
2025-01-10 17:47:03,656 - INFO - Page 1: Retrieved 1000 transactions.
2025-01-10 17:47:05,074 - INFO - Page 2: Retrieved 543 transactions.
2025-01-10 17:47:05,958 - WARNING - API request failed on page 3: Request failed: No transactions found
2025-01-10 17:47:05,958 - INFO - Final dataset contains 1543 unique transactions.
2025-01-10 17:47:05,992 - INFO - Data successfully saved to C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result\erc20_transfers.csv
2025-01-10 17:47:05,992 - INFO - Transfers saved to C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result


⚠️ High Alert: Transaction Hash 0x0004071f5d13be19b2fd2d8fcff09d1335b7b5e2a1e5ff71d45f9a571c8bb31c contains 3 records:
               dateTime  blockNumber   timeStamp  \
525 2024-11-19 09:41:59     21220948  1732009319   
526 2024-11-19 09:41:59     21220948  1732009319   
527 2024-11-19 09:41:59     21220948  1732009319   

                                                  hash  nonce  \
525  0x0004071f5d13be19b2fd2d8fcff09d1335b7b5e2a1e5...   4502   
526  0x0004071f5d13be19b2fd2d8fcff09d1335b7b5e2a1e5...   4502   
527  0x0004071f5d13be19b2fd2d8fcff09d1335b7b5e2a1e5...   4502   

                                             blockHash  \
525  0x757033477b44737d73aa6835ee85343cfc2434732f7b...   
526  0x757033477b44737d73aa6835ee85343cfc2434732f7b...   
527  0x757033477b44737d73aa6835ee85343cfc2434732f7b...   

                                           from  \
525  0x0000000000000000000000000000000000000000   
526  0x2c89a2ee92b9870f55989b4132a58c0e85222d86   
527  0x00000688768803bbd4

In [3]:
TOP_ACCOUNTS_DF = pd.read_csv(OUTPUT_FILE + "/top_accounts.csv")
transfers_df = pd.read_csv(OUTPUT_FILE + "/erc20_transfers.csv")
output_file = OUTPUT_FILE + "address_counts_with_name_tags.csv"

process_and_save_address(transfers_df, TOP_ACCOUNTS_DF, output_file)

100%|██████████| 96/96 [01:28<00:00,  1.08it/s]

Data has been save to C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/resultaddress_counts_with_name_tags.csv


In [4]:
address_counts = pd.read_csv(OUTPUT_FILE + "/address_counts_with_name_tags.csv")
address_threshold = 10
important_addresses = address_counts[address_counts["total_value_count"] > address_threshold]
important_addresses = important_addresses.sort_values(by="total_value_count", ascending=False)
# important_addresses = important_addresses[important_addresses["address"] != ADDRESS.lower()]

print(important_addresses)


                                       address  from_count  to_count  \
11  0x5be9a4959308a0d0c7bc0870e319314d8d957dbb         112      4151   
31  0xe217e15b3c19cc0427f9492dc3bcfe8220afad10        4097         1   
20  0x9008d19f58aabd9ed0d60971565aa8510560ab41          40        73   

    total_value_count  is_in_top_accounts                      name_tag  
11               4263               False       World Liberty: Multisig  
31               4098               False   TransparentUpgradeableProxy  
20                113               False  CoW Protocol: GPv2Settlement  
